In [1]:
''' Conjunto de funcoes para calcular dias uteis e feriados feriados
Dias da semana segundo a norma ISO 8601
weekdays = {1: 'Segunda-feira', 2: 'Terça-feira', 3: 'Quarta-feira',
            4: 'Quinta-feira', 5: 'Sexta-Feira', 6: 'Sabado', 7: 'Domingo'}
'''


from datetime import datetime, timedelta
from dateutil.easter import easter

import calendar

import pandas as pd

# o primeiro dia da semana aqui definido é so para poder calcular alguns feriados moveis
cld = calendar.Calendar(firstweekday=calendar.MONDAY)

In [2]:
distritos = {  # feriados de distritos e municipios de portugal
    'aveiro': {
        'agueda': ('p', 50), 'albergaria-a-velha': ('c', 8, 3, 0), 'anadia': ('p', 39), 'arouca': ('d', 5, 2), 'aveiro': ('d', 5, 12),
        'castelo-de-paiva': ('d', 6, 24), 'espinho': ('d', 6, 16), 'estarreja': ('d', 6, 13), 'santa-maria-da-feira': ('d', 1, 20),
        'ilhavo': ('p', 1), 'mealhada': ('p', 39), 'murtosa': ('d', 9, 8), 'oliveira-de-azemeis': ('c', 8, 2, 0),
        'oliveira-do-bairro': ('p', 39), 'ovar': ('d', 7, 25), 'sao-joao-da-madeira': ('d', 10, 11), 'sever-do-vouga': ('d', 9, 21),
        'vagos': ('p', 50), 'vale-de-cambra': ('d', 6, 13)
    },
    'beja': {
        'aljustrel': ('d', 6, 13), 'almodovar': ('d', 6, 24), 'alvito': ('p', 39), 'barrancos': ('d', 8, 28), 'beja': ('p', 39),
        'castro-verde': ('d', 6, 29), 'cuba': ('p', 1), 'ferreira-do-alentejo': ('d', 3, 5), 'mertola': ('d', 6, 24),
        'moura': ('d', 6, 24), 'odemira': ('d', 9, 8), 'ourique': ('d', 9, 8), 'serpa': (None,), 'vidigueira': ('p', 39)
    },
    'braga': {
        'amares': ('d', 6, 13), 'barcelos': ('d', 5, 3), 'braga': ('d', 6, 24), 'cabeceiras-de-basto': ('d', 9, 29),
        'celorico-de-basto': ('d', 7, 25), 'esposende': ('d', 8, 19), 'fafe': ('d', 5, 16), 'guimaraes': ('d', 6, 24),
        'povoa-de-lanhoso': ('d', 3, 19), 'terras-de-bouro': ('d', 10, 20), 'vieira-do-minho': ('c', 10, 1, 0),
        'vila-nova-de-famalicao': ('d', 6, 13), 'vila-verde': ('d', 6, 13), 'vizela': ('d', 3, 19)
    },
    'braganca': {
        'alfandega-da-fe': ('d', 6, 29), 'braganca': ('d', 8, 22), 'carrazeda-de-ansiaes': (None,), 'freixo-de-espada-a-cinta': ('p', 1),
        'macedo-de-cavaleiros': ('d', 6, 29), 'miranda-do-douro': ('d', 7, 10), 'mirandela': ('d', 5, 25), 'mogadouro': ('d', 10, 15),
        'torre-de-moncorvo': ('d', 3, 19), 'vila-flor': ('d', 8, 24), 'vimioso': ('d', 8, 10), 'vinhais': ('d', 5, 20)
    },
    'castelo-branco': {
        'belmonte': ('d', 4, 26), 'castelo-branco': ('p', 9), 'covilha': ('d', 10, 20), 'fundao': ('d', 9, 15),
        'idanha-a-nova': ('p', 15), 'oleiros': ('c', 8, 2, 0), 'penamacor': ('p', 1), 'proenca-a-nova': ('d', 6, 13),
        'serta': ('d', 6, 24), 'vila-de-rei': ('d', 9, 19), 'vila-velha-de-rodao': ('c', 8, 4, 0)
    },
    'coimbra': {
        'arganil': ('d', 9, 7), 'cantanhede': ('d', 7, 25), 'coimbra': ('d', 7, 4), 'condeixa-a-nova': ('d', 7, 24),
        'figueira-da-foz': ('d', 6, 24), 'gois': ('d', 8, 13), 'lousa': ('d', 6, 24), 'mira': ('d', 7, 25),
        'miranda-do-corvo': ('d', 6, 1), 'montemor-o-velho': ('d', 9, 8), 'oliveira-do-hospital': ('d', 10, 7), 'pampilhosa-da-serra': ('d', 4, 10),
        'penacova': ('d', 7, 17), 'penela': ('d', 9, 29), 'soure': ('d', 9, 21), 'tabua': ('d', 4, 10), 'vila-nova-de-poiares': ('d', 1, 13)
    },
    'evora': {
        'alandroal': ('p', 8), 'arraiolos': ('p', 39), 'borba': ('p', 1), 'estremoz': ('p', 39), 'evora': ('d', 6, 29),
        'montemor-o-novo': ('d', 3, 8), 'mora': ('p', 1), 'mourao': ('d', 2, 2), 'portel': ('p', 1), 'redondo': ('p', 1),
        'reguengos-de-monsaraz': ('d', 6, 13), 'vendas-novas': ('d', 9, 7), 'viana-do-alentejo': ('d', 1, 13), 'vila-vicosa': ('d', 8, 16)
    },
    'faro': {
        'albufeira': ('d', 8, 20), 'alcoutim': ('c', 9, 1, 4), 'aljezur': ('d', 8, 29), 'castro-marim': ('d', 6, 24),
        'faro': ('d', 9, 7), 'lagoa': ('d', 9, 8), 'lagos': ('d', 10, 27), 'loule': ('p', 39), 'monchique': ('p', 39), 'olhao': ('d', 6, 16),
        'portimao': ('d', 12, 11), 'quarteira': ('p', 39), 'sao-bras-de-alportel': ('d', 6, 1), 'silves': ('d', 9, 3),
        'tavira': ('d', 6, 24), 'vila-do-bispo': ('d', 1, 22), 'vila-real-de-santo-antonio': ('d', 5, 13)
    },
    'guarda': {
        'aguiar-da-beira': ('d', 2, 10), 'almeida': ('d', 7, 2), 'celorico-da-beira': ('d', 5, 23), 'figueira-de-castelo-rodrigo': ('d', 7, 7),
        'fornos-de-algodres': ('d', 9, 29), 'gouveia': ('c', 8, 2, 0), 'guarda': ('d', 11, 27), 'manteigas': ('d', 3, 4),
        'meda': ('d', 11, 11), 'pinhel': ('d', 8, 25), 'sabugal': ('p', 8), 'seia': ('d', 7, 3), 'trancoso': ('d', 5, 29),
        'vila-nova-de-foz-coa': ('d', 5, 21)
    },
    'leiria': {
        'alcobaca': ('d', 8, 20), 'alvaiazere': ('d', 6, 13), 'ansiao': ('p', 39), 'batalha': ('d', 8, 14), 'bombarral': ('d', 6, 29),
        'caldas-da-rainha': ('d', 5, 15), 'castanheira-de-pera': ('d', 7, 4), 'figueiro-dos-vinhos': ('d', 6, 24), 'leiria': ('d', 5, 22),
        'marinha-grande': ('p', 39), 'nazare': ('d', 9, 8), 'obidos': ('d', 1, 11), 'pedrogao-grande': ('d', 7, 24),
        'peniche': ('c', 8, 1, 0), 'pombal': ('d', 11, 11), 'porto-de-mos': ('d', 6, 29)
    },
    'lisboa': {
        'alenquer': ('p', 39), 'arruda-dos-vinhos': ('p', 39), 'azambuja': ('p', 39), 'cadaval': ('d', 1, 13),
        'cascais': ('d', 6, 13), 'lisboa': ('d', 6, 13), 'loures': ('d', 8, 26), 'lourinha': ('d', 6, 24), 'mafra': ('p', 39),
        'oeiras': ('d', 6, 7), 'sintra': ('d', 6, 29), 'sobral-de-monte-agraco': ('p', 39), 'torres-vedras': ('d', 11, 11),
        'vila-franca-de-xira': ('p', 39), 'amadora': ('d', 9, 11), 'odivelas': ('d', 11, 19)
    },
    'portalegre': {
        'alter-do-chao': ('p', 39), 'arronches': ('d', 6, 24), 'avis': ('p', 1), 'campo-maior': ('p', 1), 'castelo-de-vide': ('p', 1),
        'crato': ('p', 1), 'elvas': ('d', 1, 14), 'fronteira': ('d', 4, 6), 'gaviao': ('d', 11, 23), 'marvao': ('d', 9, 8), 'monforte': ('p', 8),
        'nisa': ('p', 1), 'ponte-de-sor': ('p', 1), 'portalegre': ('d', 5, 23), 'sousel': ('p', 1)
    },
    'porto': {
        'amarante': ('d', 7, 8), 'baiao': ('d', 8, 24), 'felgueiras': ('d', 6, 29), 'gondomar': ('c', 10, 1, 0),
        'lousada': ('e', 7), 'maia': ('c', 7, 2, 0), 'marco-de-canaveses': ('d', 9, 8), 'matosinhos': ('p', 51),
        'pacos-de-ferreira': ('d', 11, 6), 'paredes': ('c', 7, 3, 0), 'penafiel': ('d', 11, 11), 'porto': ('d', 6, 24),
        'povoa-de-varzim': ('d', 6, 29), 'santo-tirso': ('d', 7, 11), 'valongo': ('d', 6, 24), 'vila-do-conde': ('d', 6, 24),
        'vila-nova-de-gaia': ('d', 6, 24), 'trofa': ('d', 11, 19)
    },
    'santarem': {
        'abrantes': ('d', 6, 14), 'alcanena': ('p', 39), 'almeirim': ('p', 39), 'alpiarca': ('d', 4, 2), 'benavente': ('p', 39), 'cartaxo': ('p', 39),
        'chamusca': ('p', 39), 'constancia': ('p', 1), 'coruche': ('d', 8, 17), 'entroncamento': ('d', 11, 24), 'fatima': ('d', 5, 13),
        'ferreira-do-zezere': ('d', 6, 13), 'golega': ('p', 39), 'macao': ('p', 1), 'rio-maior': ('d', 11, 6),
        'salvaterra-de-magos': ('p', 39), 'santarem': ('d', 3, 19), 'sardoal': ('d', 9, 22), 'tomar': ('d', 3, 1), 'torres-novas': ('p', 39),
        'vila-nova-da-barquinha': ('d', 6, 13), 'ourem': ('d', 6, 20)
    },
    'setubal': {
        'alcacer-do-sal': ('d', 6, 24), 'alcochete': ('d', 6, 24), 'almada': ('d', 6, 24), 'barreiro': ('d', 6, 28),
        'grandola': ('d', 10, 22), 'moita': ('c', 9, 2, 1), 'montijo': ('d', 6, 29), 'palmela': ('d', 6, 1),
        'santiago-do-cacem': ('d', 7, 25), 'seixal': ('d', 6, 29), 'sesimbra': ('d', 5, 4), 'setubal': ('d', 9, 15), 'sines': ('d', 11, 24)
    },
    'viana-do-castelo': {
        'arcos-de-valdevez': ('d', 7, 11), 'caminha': ('p', 1), 'melgaco': ('p', 39), 'moncao': ('d', 3, 12),
        'paredes-de-coura': ('d', 8, 10), 'ponte-da-barca': ('d', 8, 24), 'ponte-de-lima': ('d', 9, 20), 'valenca': ('d', 2, 18),
        'viana-do-castelo': ('d', 8, 20), 'vila-nova-de-cerveira': ('d', 10, 1)
    },
    'vila-real': {
        'alijo': ('d', 11, 11), 'boticas': ('d', 11, 6), 'chaves': ('d', 7, 8), 'mesao-frio': ('d', 11, 30), 'mondim-de-basto': ('d', 7, 25),
        'montalegre': ('d', 6, 9), 'murca': ('d', 5, 8), 'peso-da-regua': ('d', 8, 16), 'ribeira-de-pena': ('d', 8, 16),
        'sabrosa': ('d', 9, 8), 'santa-marta-de-penaguiao': ('d', 1, 13), 'valpacos': ('d', 11, 6), 'vila-pouca-de-aguiar': ('d', 6, 22), 'vila-real': ('d', 6, 13)
    },
    'viseu': {
        'armamar': ('d', 6, 24), 'carregal-do-sal': ('c', 7, 3, 0), 'castro-daire': ('d', 6, 29), 'cinfaes': ('d', 6, 24),
        'lamego': ('d', 9, 8), 'mangualde': ('d', 9, 8), 'moimenta-da-beira': ('d', 6, 24), 'mortagua': ('p', 39),
        'nelas': ('d', 6, 24), 'oliveira-de-frades': ('d', 10, 7), 'penalva-do-castelo': ('d', 8, 25), 'penedono': ('d', 6, 29),
        'resende': ('d', 9, 29), 'santa-comba-dao': ('p', 39), 'sao-joao-da-pesqueira': ('d', 6, 24), 'sao-pedro-do-sul': ('d', 6, 29),
        'satao': ('d', 8, 20), 'sernancelhe': ('d', 5, 3), 'tabuaco': ('d', 6, 24), 'tarouca': ('d', 9, 29), 'tondela': ('d', 9, 16),
        'vila-nova-de-paiva': ('d', 3, 2), 'viseu': ('d', 9, 21), 'vouzela': ('d', 5, 14)
    },
    'madeira': {
        'santa-cruz': ('d', 1, 15), 'ponta-do-sol': ('d', 9, 8), 'porto-moniz': ('d', 7, 22), 'ribeira-brava': ('d', 6, 29),
        'calheta': ('d', 6, 24), 'porto-santo': ('d', 6, 24), 'santana': ('d', 5, 25), 'sao-vicente': ('d', 1, 22),
        'funchal': ('d', 8, 21), 'camara-de-lobos': ('d', 10, 4), 'machico': ('d', 10, 9)
    },
    'acores': {
        'sao-roque-do-pico': ('d', 8, 16), 'madalena': ('d', 7, 22), 'ribeira-grande': ('d', 6, 29), 'lajes-do-pico': ('d', 6, 29),
        'angra-do-heroismo': ('d', 6, 24), 'horta': ('d', 6, 24), 'santa-cruz-das-flores': ('d', 6, 24), 'vila-franca-do-campo': ('d', 6, 24),
        'vila-do-porto': ('d', 6, 24), 'praia-da-vitoria': ('d', 6, 20), 'vila-nova-do-corvo': ('d', 6, 20), 'lagoa': ('d', 4, 11),
        'velas': ('d', 4, 23), 'nordeste': ('d', 7, 18), 'calheta': ('d', 11, 25), 'ponta-delgada': ('p', 36),
        'povoacao': ('p', 61), 'santa-cruz-da-graciosa': ('c', 8, 2, 0),
    }
}

In [3]:
def get_leap_days(start, end):
    ''' Calcular os anos bissextos para o intervalo '''
    
    return [datetime(i, 2 , 29) for i in range(start, end+1, 1) if calendar.isleap(i)]

get_leap_days(2010, 2020)

[datetime.datetime(2012, 2, 29, 0, 0),
 datetime.datetime(2016, 2, 29, 0, 0),
 datetime.datetime(2020, 2, 29, 0, 0)]

In [4]:
def get_leap_days(start, end):
    ''' Calcular os anos bissextos para o intervalo '''
    
    return {i: (True if calendar.isleap(i) else False) for i in range(start, end+1, 1)}

get_leap_days(2010, 2020)

{2010: False,
 2011: False,
 2012: True,
 2013: False,
 2014: False,
 2015: False,
 2016: True,
 2017: False,
 2018: False,
 2019: False,
 2020: True}

In [5]:
def get_feriados_nacionais(ano):
    ''' Devolve uma lista de objectos datetime dos feriados nacionais para o ano '''

    _p = easter(ano)  # isto da a data como objecto de date e nao datetime
    pascoa = datetime(_p.year, _p.month, _p.day)

    feriados_nacionais = [
        datetime(ano, 1, 1),    # ano novo
        pascoa+timedelta(days=-47),  # carnaval
        pascoa+timedelta(days=-2),   # sexta-feira santa
        pascoa,  # pascoa
        datetime(ano, 4, 25),   # 25 de abril
        datetime(ano, 5, 1),    # dia do trabalhador
        datetime(ano, 6, 10),   # dia de portugal
        pascoa+timedelta(days=60),  # corpo de deus
        datetime(ano, 8, 15),   # assuncao maria
        datetime(ano, 10, 5),   # implatacao da republica
        datetime(ano, 11, 1),   # todos os santos
        datetime(ano, 12, 1),   # restauracao da independencia
        datetime(ano, 12, 8),   # imaculada conceicao
        datetime(ano, 12, 25),  # natal
    ]

    return feriados_nacionais

# _feriados = []
# for i in range(2011, 2021, 1):
#     _feriados += get_feriados_nacionais(i)
    
# _feriados

In [14]:
def get_multi_feriados_nacionais(start, end):
    ''' Devolve lista com feriados nacionais para varios anos '''
    
    feriados_nacionais = []
    for i in range(start, end+1, 1):
        feriados_nacionais += get_feriados_nacionais(i)
    
    return feriados_nacionais


get_multi_feriados_nacionais(2019, 2100)

[datetime.datetime(2019, 1, 1, 0, 0),
 datetime.datetime(2019, 3, 5, 0, 0),
 datetime.datetime(2019, 4, 19, 0, 0),
 datetime.datetime(2019, 4, 21, 0, 0),
 datetime.datetime(2019, 4, 25, 0, 0),
 datetime.datetime(2019, 5, 1, 0, 0),
 datetime.datetime(2019, 6, 10, 0, 0),
 datetime.datetime(2019, 6, 20, 0, 0),
 datetime.datetime(2019, 8, 15, 0, 0),
 datetime.datetime(2019, 10, 5, 0, 0),
 datetime.datetime(2019, 11, 1, 0, 0),
 datetime.datetime(2019, 12, 1, 0, 0),
 datetime.datetime(2019, 12, 8, 0, 0),
 datetime.datetime(2019, 12, 25, 0, 0),
 datetime.datetime(2020, 1, 1, 0, 0),
 datetime.datetime(2020, 2, 25, 0, 0),
 datetime.datetime(2020, 4, 10, 0, 0),
 datetime.datetime(2020, 4, 12, 0, 0),
 datetime.datetime(2020, 4, 25, 0, 0),
 datetime.datetime(2020, 5, 1, 0, 0),
 datetime.datetime(2020, 6, 10, 0, 0),
 datetime.datetime(2020, 6, 11, 0, 0),
 datetime.datetime(2020, 8, 15, 0, 0),
 datetime.datetime(2020, 10, 5, 0, 0),
 datetime.datetime(2020, 11, 1, 0, 0),
 datetime.datetime(2020, 12, 

In [7]:
def get_feriados_municipais(ano, cidade):
    ''' calcular o dia do feriado para a cidade e para o ano dado '''

    _p = easter(ano)  # isto da data como objecto de date e nao datetime
    pascoa = datetime(_p.year, _p.month, _p.day)

    def _get_cld(ano, mes, semana, dia):  # mesmo problema com a pascoa
        d = cld.monthdatescalendar(ano, mes)[semana][dia]
        return datetime(d.year, d.month, d.day)

    # obter codigo para calcular o dia do feriado
    for k, v in distritos.items():
        if cidade in v:
            codigo = v[cidade]
        else:
            pass

    check = codigo[0]
    if check == 'd':  # dias fixos
        feriado = datetime(ano, codigo[1], codigo[2])
    elif check == 'p':  # dias dependentes da pascoa
        feriado = pascoa+timedelta(days=codigo[1])
    elif check == 'c':  # feriados tipo 'segunda depois de terceiro domingo...'
        feriado = _get_cld(ano, codigo[1], codigo[2], codigo[3])
    elif check == 'e':  # caso especial da lousada
        feriado = datetime(ano, 7, max(w[-1] for w in calendar.monthcalendar(ano, 7)))+timedelta(days=1)
    else:
        feriado = None

    return feriado

# for k, v in distritos.items():
#     for _k in v.keys():
#         print(k, _k, get_feriados_municipais(2020, _k))
# print(get_feriados_municipais(2020, 'leiria'))

In [8]:
def get_feriados_zona(ano, distrito=None, municipio=None):
    ''' Obter lista de feriados municipais para um distrito, muncipio ou
    lista de distritos/municipios
    '''

    feriados = []

    if distrito is not None:
        dist = distrito if isinstance(distrito, list) else [distrito]
        for d in dist:
            for m in distritos[d].keys():
                feriado = get_feriados_municipais(ano, m)
                if feriado not in feriados:
                    feriados.append(feriado)

    if municipio is not None:
        muni = municipio if isinstance(municipio, list) else [municipio]
        for m in muni:
            feriado = get_feriados_municipais(ano, m)
            if feriado not in feriados:
                feriados.append(feriado)

    return feriados

# get_feriados_zona(2018, distrito='lisboa')

In [9]:
def get_days_range(target, date_format='%Y-%m-%d'):
    ''' Calcular dias no intervalo das datas ou para a semana
    inputs podem ser:
    - (start, end) tuple com data de inicio ou fim ex. ('2020-4-6', '2020-4-12')
    - 'year-week' str com ano e semana alvo ex. '2020-15'
    '''

    if isinstance(target, tuple):
        start = datetime.strptime(target[0], date_format)-timedelta(days=1)
        end = datetime.strptime(target[1], date_format)
    else:  # atencao ao formato iso; calcula para uma determinada semana
        start = datetime.strptime(target+'-1', "%G-%V-%u")-timedelta(days=1)
        end = datetime.strptime(target+'-7', "%G-%V-%u")

    return pd.Series(index=[start + timedelta(x + 1) for x in range((end - start).days)], dtype='int32')

# semana 15
# print(get_days_range('2020-15'))
# print(get_days_range(('2020-4-6', '2020-4-12')))

In [10]:
def calc_days_type(target, feriados=False, date_format='%Y-%m-%d', **kwargs):
    ''' Para o intervalo de tempo alvo classificar os dias de acordo com o codigo:
        - dia util: 0
        - fim de semana: 1 
        - feriado nacional: 2
        - feriado municipal: 3
    '''

    target_days = get_days_range(target)
    
    # calcular uma unica vez os feriados disponiveis
    f_nacionais, f_municipais = [], []
    if feriados:
        # obter os anos no intervalo de tempo dado
        years =  [y.year for y in target_days.groupby(pd.Grouper(freq='Y')).sum().index.unique()]
        distrito = kwargs['distrito'] if 'distrito' in kwargs else None
        municipio = kwargs['municipio'] if 'municipio' in kwargs else None
        for y in years:
            f_nacionais += get_feriados_nacionais(y)
            f_municipais += get_feriados_zona(y, distrito=distrito,  municipio=municipio)

    for day in target_days.index:
        weekday = day.isoweekday()
        target_days[day] = 0 if weekday < 6 else 1
        if feriados:
            if day in f_municipais:
                target_days[day] = 3
            if day in f_nacionais:
                target_days[day] = 2
                
    return target_days

export = calc_days_type(('2018-12-31', '2021-1-2'), feriados=True)
export
# export.to_csv(r'C:\Users\859494\Desktop\dias_uteis_2019_2020.csv')

2018-12-31    0
2019-01-01    2
2019-01-02    0
2019-01-03    0
2019-01-04    0
             ..
2020-12-29    0
2020-12-30    0
2020-12-31    0
2021-01-01    2
2021-01-02    1
Length: 734, dtype: int32

In [11]:
def get_dias_uteis(periodo, group=False, feriados=False, date_format='%Y-%m-%d', **kwargs):
    ''' Classificar dias em uteis e nao para um determinado intervalo de tempo
     - dia util : 1
     - fim de semana/feriado : 0
    
    Se as datas nao forem agrupadas ira retornar o sumatorio dos dias uteis
    
    : params :
    periodo : intervalo de tempo para o qual calcular o os dias uteis
    group : se False devolve o total, se selecionar agrupar devolve o numero de dias uteis
        - 'W' : agrupar por semanas
        - 'M' : agrupar por mes
        - 'Y' : agurpar por ano
    feriados : descontar feriados
    date_format : formato da data de input
    kwargs : opcao de incluir feriados de distrito ou municipio
        - distrito : ao indicar distrito isto inclui os feriados de todos os
                     concelhos pertencentes ao distrito
                     pode ser str ou uma list de strings para varios distritos
        - municipio : incluir feriados de um certo municipio
                     pode ser str ou uma list de strings para varios concelhos
    '''
    
    by_day = calc_days_type(periodo, feriados, date_format, **kwargs)
    
    if not group:  # senao agrupar calcula e devolve a soma
        dias_total = by_day.count()
        dias_uteis = by_day[by_day == 0].count()
        dias_f_nacionais = by_day[by_day == 2].count()
        dias_f_municipais = by_day[by_day == 3].count()

        results = (f'Total dias: {dias_total} - Dias uteis: {dias_uteis}'
                   f' - Feriados Nacionais: {dias_f_nacionais}'
                   f' - Feriados Municipais: {dias_f_municipais}')
    else:
        dias_uteis = by_day[by_day == 0]  # selecionar so dias uteis
        results = dias_uteis.groupby(pd.Grouper(freq=group)).count()
    
    return results

get_dias_uteis(('2019-12-1', '2020-12-31'), group='D', feriados=True)

2019-12-02    1
2019-12-03    1
2019-12-04    1
2019-12-05    1
2019-12-06    1
             ..
2020-12-27    0
2020-12-28    1
2020-12-29    1
2020-12-30    1
2020-12-31    1
Freq: D, Length: 396, dtype: int64

In [12]:
def class_distancia_fds(df):
    ''' Classificar os dias da semana conforme a distancia ao fim de semana
    
    fim de semana : 0 ;
    segunda/sexta : 3 ;
    terca/quinta  : 2 ;
    quarta        : 1 ;
    
    dias da semana segundo a norma ISO 8601 -> datetime.isoweekday()
    1: 'Segunda-feira', 2: 'Terça-feira', 3: 'Quarta-feira',
    4: 'Quinta-feira', 5: 'Sexta-Feira', 6: 'Sabado', 7: 'Domingo'
    '''
    
    df = df.assign(distancia=0)
    
    distancias = {1: 3, 2: 2, 3: 1, 4: 2, 5: 3, 6: 0, 7: 0}
    
    for i in df.index:
        df.at[i, 'distancia'] = distancias[datetime.isoweekday(i)]
        
    return df

In [13]:
def get_feriados(intervalo, distance=False, multiply=False, group='B', **kwargs):
    ''' Classificar dias em dias uteis ou feriados e calcular a distancia ao fim de semana se necessario 
    - dias_uteis : 0
    - feriados : 1
   
    Esta funcao e um wrapper de varias funcoes para facilitar o calculo de feriados para usar como variavel
    externa para introduzir em modelo
    
    : params :
    intervalo (tuple): intervalo de tempo para calcular os feriados/dias uteis, aceita um tuple de strings de data
                       no formato YYYY-mm-dd
    distance (bool): calcular a distancia ao fim de semana ou nao
    multiply (bool): multiplicar a distancia pela classe do dia
    group (str): ver func get_dias_uteis: aqui so aceita esta reduzido a 'B' ou 'D'
    kwargs : ver func get_dias_uteis
    
    : returns :
    df: dataframe com a classificacao dos dias, se multiplicar devolve so coluna do feriado senao devolve o feriado e 
        a distancia
    '''
    
    df = (get_dias_uteis(intervalo, group=group, feriados=True)
          .to_frame()
          .rename(columns={0: 'feriado'})
          .replace({0: 1, 1: 0})
         )
    if distance:
        df = class_distancia_fds(df)
        if multiply:
            df = (df['feriado']*df['distancia']).to_frame().rename(columns={0: 'feriado'})
    
    return df

get_feriados(('2010-01-01', '2020-01-01'), distance=True, multiply=True).head(n=60)

,feriado
2010-01-04,0
2010-01-05,0
2010-01-06,0
2010-01-07,0
2010-01-08,0
2010-01-11,0
2010-01-12,0
2010-01-13,0
2010-01-14,0
2010-01-15,0
